In [1]:
# Importer le module librosa
import matplotlib.pyplot as plt
import librosa.display
import librosa
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
# Définir une fonction pour lire un fichier audio
def read_audio_file(file):
    # Ouvrir le fichier validation_liste en mode lecture
    with open(file, "r") as f:
        # Créer une liste vide pour stocker les données audio
        audio_data = []

        # Créer une liste vide pour stocker les étiquettes
        labels = []
        
        # Parcourir chaque ligne du fichier
        for line in f:
            # Supprimer le caractère de fin de ligne
            line = line.strip()
            # Charger le fichier audio avec librosa
            y, sr = librosa.load(line, sr=16000)
            # Ajouter le signal sonore et le taux d'échantillonnage à la liste
            audio_data.append((y, sr))
            # Ajouter l'étiquette à la liste
            labels.append(line.split('/')[0])

    return audio_data, labels

# Charger le fichier audio
audio_data, labels = read_audio_file("validation_list.txt")

# Créer un encodeur d'étiquettes
label_encoder = LabelEncoder()

# Adapter l'encodeur aux étiquettes pour créer une correspondance entre les noms de classe et les entiers
label_encoder.fit(labels)

# Transformer les étiquettes en entiers
labels = label_encoder.transform(labels)

In [3]:
# Sélectionner les MFCC pour les 50 premières trames (1 seconde)
stride = 320 / 16000
one_second_frames = int(1 / stride)

def extract_mfcc_features(audio_data, sr=16000, n_mfcc=40, n_fft=1024, hop_length=320):
    features = []
    index_to_remove = []
    for i, (audio, _) in enumerate(audio_data):
        # Extraire les MFCC à partir du signal audio
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)

        # Sélectionner les MFCC pour les 49 premières trames (1 seconde)
        mfccs_one_second = mfccs[:, :one_second_frames]

        if mfccs_one_second.shape[1] != one_second_frames:
            index_to_remove.append(i)
            continue
        
        mfccs = np.reshape(mfccs_one_second, (40 * one_second_frames,))

        # Transposer les MFCC pour correspondre à l'orientation attendue (temps x caractéristiques)
        features.append(mfccs.T)
    return features, index_to_remove


In [4]:
mfcc_features, index_to_remove = extract_mfcc_features(audio_data)
Y = np.delete(labels, index_to_remove)

In [ ]:
# Chargement des données test
test_audio_data, test_labels = read_audio_file("testing_list.txt")

# Extraire les caractéristiques des signaux audio
test_mfcc_features, test_index_to_remove = extract_mfcc_features(test_audio_data)
test_mfcc_labels = np.delete(label_encoder.transform(test_labels), test_index_to_remove)

In [ ]:
X_train =np.array(mfcc_features)

# Créer un objet StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Utiliser la cross-validation pour évaluer le modèle
# Créer un classifieur LogisticRegression multi-classe
logistic_regression = LogisticRegression(max_iter=1000, multi_class="multinomial")

# Utiliser la cross-validation pour évaluer le modèle
scores = cross_val_score(logistic_regression, X_train, mfcc_labels, cv=5)

# Afficher les scores de la cross-validation
print("Scores de la cross-validation : ", scores)

# Entraîner le modèle
logistic_regression.fit(X_train, mfcc_labels)

# Prédire les étiquettes des données test
y_pred = logistic_regression.predict(X_test)

# Afficher le rapport de classification
print(classification_report(test_mfcc_labels, y_pred))

# Afficher la précision
print("Précision : ", accuracy_score(test_mfcc_labels, y_pred))



In [5]:
X = np.array(mfcc_features)

# Normalisation des caractéristiques
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Séparation des données
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42, stratify=Y)

# Création et entraînement du SVM linéaire
clf = LogisticRegression(max_iter=1000)  # Par défaut, il gère la classification multiclasse

#cross-validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Scores de la cross-validation : ", scores)

clf.fit(X_train, y_train)

# Évaluation du modèle
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

Scores de la cross-validation :  [0.32342778 0.34830684 0.33586731 0.32135453 0.33195021]
              precision    recall  f1-score   support

           0       0.34      0.36      0.35        28
           1       0.26      0.25      0.26        36
           2       0.42      0.34      0.38        32
           3       0.28      0.23      0.25        31
           4       0.00      0.00      0.00        34
           5       0.22      0.24      0.23        70
           6       0.43      0.44      0.43        64
           7       0.34      0.31      0.33        68
           8       0.53      0.43      0.48        23
           9       0.77      0.37      0.50        27
          10       0.32      0.36      0.34        70
          11       0.25      0.28      0.27        68
          12       0.33      0.19      0.24        37
          13       0.50      0.40      0.44        35
          14       0.33      0.25      0.29        24
          15       0.18      0.16      0.17  